In [18]:
import pandas as pd
import numpy as np
import random

random.seed(824)

dataset_dir = "../datasets/ml-25m"

movies_df = pd.read_csv(dataset_dir + "/movies.csv")
movies_df = movies_df.set_index("movieId")
ratings_df = pd.read_csv(dataset_dir + "/ratings.csv")
tags_df = pd.read_csv(dataset_dir + "/tags.csv")

print('Setup complete!')

Setup complete!


In [19]:
users = set(ratings_df["userId"])
len(users)

162541

In [20]:
#reducing dataset size
for i in range(len(users)):
    split = random.choices([0, 1], weights=(97,3), k=1)[0]
    if split == 0:
        users.remove(i+1)
print(len(users))

4891


In [21]:
ratings_df = ratings_df[ratings_df["userId"].map(lambda id: id in users)]
ratings_df

,userId,movieId,rating,timestamp
4446,28,32,5.0,951086359
4447,28,253,5.0,951086752
4448,28,260,5.0,951086359
4449,28,292,3.0,951086104
4450,28,316,5.0,951086624
...,...,...,...,...
24982751,162473,6242,3.5,1060836405
24982752,162473,6310,3.5,1060227604
24982753,162473,6345,4.0,1060227365
24982754,162473,6396,2.5,1063989056


In [22]:
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
209157,We (2018),Drama
209159,Window of the Soul (2001),Documentary
209163,Bad Poems (2018),Comedy|Drama


In [23]:
ratings_df = ratings_df.drop("timestamp", axis=1)
ratings_df

,userId,movieId,rating
4446,28,32,5.0
4447,28,253,5.0
4448,28,260,5.0
4449,28,292,3.0
4450,28,316,5.0
...,...,...,...
24982751,162473,6242,3.5
24982752,162473,6310,3.5
24982753,162473,6345,4.0
24982754,162473,6396,2.5


In [24]:
#retain all movies with at least 5 ratings
def add_valid_movies(row):
    if row["counts"] >= 5:
        movieIds.add(row["movieId"])

movieIds = set()
movie_rating_counts = ratings_df["movieId"].value_counts().rename_axis('movieId').reset_index(name='counts')
movie_rating_counts.apply(add_valid_movies, axis=1)
print(len(movieIds))

8804


In [25]:
movies_df = movies_df[movies_df.index.map(lambda id: id in movieIds)]
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
203222,The Lion King (2019),Adventure|Animation|Children|Drama
203519,Fast & Furious Presents: Hobbs & Shaw (2019),Action
204352,Ad Astra (2019),Adventure|Drama|Mystery|Sci-Fi|Thriller


In [26]:
genres_list = []
for index, row in movies_df.iterrows():
    try:
        genres = row['genres'].split("|")
        genres_list.extend(genres)
    except:
        genres_list.append(row["genres"])

genres_list = list(set(genres_list))

In [27]:
genres_list.remove("IMAX")
repl_idx = genres_list.remove("(no genres listed)")
for genre in genres_list: 
    print(genre)
print(len(genres_list))

Fantasy
Romance
Musical
Horror
Crime
Children
Documentary
Mystery
Animation
Adventure
War
Drama
Thriller
Film-Noir
Comedy
Sci-Fi
Western
Action
18


In [28]:
movies_df1 = movies_df.copy()
for genre in genres_list:
    movies_df1[genre] = movies_df1["genres"].map(lambda o: 1 if genre in str(o) else 0)
movies_df1

,title,genres,Fantasy,Romance,Musical,Horror,Crime,Children,Documentary,Mystery,Animation,Adventure,War,Drama,Thriller,Film-Noir,Comedy,Sci-Fi,Western,Action
movieId,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0
2,Jumanji (1995),Adventure|Children|Fantasy,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),Comedy|Romance,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203222,The Lion King (2019),Adventure|Animation|Children|Drama,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0
203519,Fast & Furious Presents: Hobbs & Shaw (2019),Action,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
204352,Ad Astra (2019),Adventure|Drama|Mystery|Sci-Fi|Thriller,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,0


In [29]:
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
203222,The Lion King (2019),Adventure|Animation|Children|Drama
203519,Fast & Furious Presents: Hobbs & Shaw (2019),Action
204352,Ad Astra (2019),Adventure|Drama|Mystery|Sci-Fi|Thriller


In [30]:
tags_df = tags_df[tags_df["movieId"].map(lambda id: id in movieIds)]

In [31]:
tags_df["tag"] = tags_df["tag"].map(lambda t: str(t).lower()) #all tags to small letters
tag_counts = tags_df["tag"].value_counts().rename_axis('tag').reset_index(name='counts')
tag_counts = tag_counts.groupby("tag").counts.agg([sum]) #merge duplicates
tag_counts

,sum
tag,
ballet school,1
breakup,1
filmes antigos,2
filmes antigos,2
mental illness,2
...,...
惊悚,1
扭曲,1
淘金记 莫声版,1


In [32]:
tag_counts = tag_counts[tag_counts["sum"] >= 20]
tag_counts = tag_counts.sort_values(["sum","tag"], ascending=[False,True]).reset_index()
tag_counts

,tag,sum
0,sci-fi,8599
1,atmospheric,6851
2,action,6490
3,comedy,5563
4,surreal,5216
...,...,...
4541,w,20
4542,walked out/didn't finish,20
4543,watch-again,20
4544,whodunnit,20


In [33]:
init_tag = np.zeros(len(movies_df))
movies_df2 = movies_df.copy()

def add_tags_as_features(tag):
    relevant_tags.add(tag)
    movies_df2[tag] = init_tag
    
relevant_tags = set()
tag_counts["tag"].apply(add_tags_as_features)
    

0       None
1       None
2       None
3       None
4       None
        ... 
4541    None
4542    None
4543    None
4544    None
4545    None
Name: tag, Length: 4546, dtype: object

In [34]:
movies_df2 = movies_df2.drop(["genres", "title"], axis=1)
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,twist ending,based on a book,visually appealing,dystopia,...,union,unsatisfying,unusual story,violence in america,violin,w,walked out/didn't finish,watch-again,whodunnit,zhang yimou
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
#around 10 minutes

def add_tags_to_movies(row):
    tag = row["tag"]
    if tag not in relevant_tags:
        return
    movieId = row["movieId"]
    movies_df2.loc[movieId, tag] += 1
    
tags_df.apply(add_tags_to_movies, axis=1)

#for index, row in tags_df.iterrows():
#    if index % 10000 == 0:
#        print(index/10000)
#    tag = row["tag"]
#    if tag not in relevant_tags:
#        continue
#    movieId = row["movieId"]
#    movies_df.loc[movieId, tag] += 1

0          None
1          None
2          None
3          None
4          None
           ... 
1093352    None
1093353    None
1093354    None
1093355    None
1093356    None
Length: 869283, dtype: object

In [36]:
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,twist ending,based on a book,visually appealing,dystopia,...,union,unsatisfying,unusual story,violence in america,violin,w,walked out/didn't finish,watch-again,whodunnit,zhang yimou
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,20.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203222,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
movies_df2["funny"]

movieId
1         30.0
2          0.0
3          1.0
4          0.0
5          0.0
          ... 
203222     1.0
203519     0.0
204352     0.0
204698     0.0
205383     0.0
Name: funny, Length: 8804, dtype: float64

In [38]:
movies_df2["max"] = movies_df2.max(axis=1)
novi = movies_df2[movies_df2["max"] > 0]
novi = novi.div(novi["max"], axis=0)
novi = novi.drop("max", axis=1)
novi

,sci-fi,atmospheric,action,comedy,surreal,funny,twist ending,based on a book,visually appealing,dystopia,...,union,unsatisfying,unusual story,violence in america,violin,w,walked out/didn't finish,watch-again,whodunnit,zhang yimou
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.008772,0.000000,0.008772,0.175439,0.0,0.263158,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.058824,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203222,0.000000,0.000000,0.000000,0.142857,0.0,0.142857,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203519,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204352,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,twist ending,based on a book,visually appealing,dystopia,...,unsatisfying,unusual story,violence in america,violin,w,walked out/didn't finish,watch-again,whodunnit,zhang yimou,max
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,20.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203222,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
203519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
204352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0


In [40]:
movies_df2 = novi
movies_df1 = movies_df1.drop(["genres", "title"], axis=1)
movies_df1

,Fantasy,Romance,Musical,Horror,Crime,Children,Documentary,Mystery,Animation,Adventure,War,Drama,Thriller,Film-Noir,Comedy,Sci-Fi,Western,Action
movieId,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0
2,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203222,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0
203519,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
204352,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,0


In [41]:
movies_df = pd.concat([movies_df1, movies_df2], axis=1)
movies_df = movies_df.dropna()
movies_df

,Fantasy,Romance,Musical,Horror,Crime,Children,Documentary,Mystery,Animation,Adventure,...,union,unsatisfying,unusual story,violence in america,violin,w,walked out/didn't finish,watch-again,whodunnit,zhang yimou
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,1,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203222,0,0,0,0,0,1,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203519,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204352,0,0,0,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
#few minutes
movies_df.to_csv("movie_features-small.csv")

In [43]:
ratings_df

,userId,movieId,rating
4446,28,32,5.0
4447,28,253,5.0
4448,28,260,5.0
4449,28,292,3.0
4450,28,316,5.0
...,...,...,...
24982751,162473,6242,3.5
24982752,162473,6310,3.5
24982753,162473,6345,4.0
24982754,162473,6396,2.5


In [44]:
moviesIds = set(movies_df.index)
len(moviesIds)

8726

In [45]:
ratings_df = ratings_df[ratings_df["movieId"].map(lambda id: id in moviesIds)]
ratings_df

,userId,movieId,rating
4446,28,32,5.0
4447,28,253,5.0
4448,28,260,5.0
4449,28,292,3.0
4450,28,316,5.0
...,...,...,...
24982749,162473,5992,4.0
24982750,162473,6027,5.0
24982751,162473,6242,3.5
24982753,162473,6345,4.0


In [53]:
users = list(set(ratings_df["userId"]))
users.sort()
user_mapping = dict()
for i in range(len(users)):
    user_mapping[users[i]] = i + 1
print(len(user_mapping))


4891


In [55]:
ratings_df["userId"] = ratings_df["userId"].map(lambda id: user_mapping[id])
ratings_df

<ipython-input-55-c8ab4f71ae13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_df["userId"] = ratings_df["userId"].map(lambda id: user_mapping[id])


,userId,movieId,rating
4446,1,32,5.0
4447,1,253,5.0
4448,1,260,5.0
4449,1,292,3.0
4450,1,316,5.0
...,...,...,...
24982749,4891,5992,4.0
24982750,4891,6027,5.0
24982751,4891,6242,3.5
24982753,4891,6345,4.0


In [56]:
ratings_df.to_csv("rating_features-small.csv")